In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
import random

In [ ]:
import os
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/drive/MyDrive/2656 ML Project/archive/new_cards_file.csv"
data = pd.read_csv(file_path)

data.head()

,name,number,arcana,suit,img,fortune_telling,keywords,Archetype,Hebrew Alphabet,Numerology,Elemental,Mythical/Spiritual,Questions to Ask,meanings.light,meanings.shadow,Astrology,Affirmation
0,The Fool,0,Major Arcana,Trump,m00.jpg,"['Watch for new projects and new beginnings', ...","['freedom', 'faith', 'inexperience', 'innocence']",The Divine Madman,Aleph/Ox/1,0 (off the scale; pure potential),Air,Adam before the fall. Christ as a wandering ho...,['What would I do if I felt free to take a lea...,"['Freeing yourself from limitation', 'Expressi...","['Being gullible and naive', 'Taking unnecessa...",NaN,NaN
1,The Magician,1,Major Arcana,Trump,m01.jpg,"['A powerful man may play a role in your day',...","['capability', 'empowerment', 'activity']",The Ego/The Self,Beth/House/2,"1 (origins, unity, seeds)",The Sun/Mercury,"Thoth, the Egyptian god of wisdom, known to th...","['What am I empowered to do?', 'How might my a...","['Taking appropriate action', 'Receiving guida...","['Inflating your own ego', 'Abusing talents', ...",NaN,NaN
2,The High Priestess,2,Major Arcana,Trump,m02.jpg,"['A mysterious woman arrives', 'A sexual secre...","['intuition', 'reflection', 'purity', 'initiat...",The Virgin/The Maiden,Gimel/Camel/3,"2 (division, debate, duality)",The Moon,"The feminine aspect of divinity, particularity...","['What might a rebel against tradition do?', ""...","['Listening to your feelings and intuitions', ...","['Being aloof', 'Obsessing on secrets and cons...",NaN,NaN
3,The Empress,3,Major Arcana,Trump,m03.jpg,"['Pregnancy is in the cards', 'An opportunity ...","['fertility', 'productivity', 'ripeness', 'nur...",The Mother,Daleth/Door/4,"3 (expression, productivity, output)",Venus,"Gaia, Mother Earth, Ishtar, DemeterÑmature, re...",['What would a concerned and capable mother do...,"['Nurturing yourself and others', 'Bearing fru...","['Overindulging', 'Being greedy', 'Smothering ...",NaN,NaN
4,The Emperor,4,Major Arcana,Trump,m04.jpg,"['A father figure arrives', 'A new employer or...","['authority', 'regulation', 'direction', 'stru...",The Father,"He[as]/Window/5, or in some decks, Tzaddi/Fish...","4 (stability, equality, persistence)",Mars/Aries,"Masculine gods, including the Hebrew God, the ...",['How does the issue of control or regulation ...,"['Exercising authority', 'Defining limits', 'D...","['Micromanaging', 'Crushing the creativity of ...",NaN,NaN


In [ ]:
import ast  # This will be used to safely evaluate string representations of lists

def process_list_data(raw_data):
    """Safely evaluate and join list-like string data into a single string."""
    try:
        list_data = ast.literal_eval(raw_data)  # Safely evaluate the string to a list
        return ' '.join(list_data)  # Join the list into a single string
    except (ValueError, SyntaxError):
        return ""

# Combine various fields into a detailed completion text
data['Prompt'] = data['name'].apply(lambda x: f"Tell me about the tarot card {x}.")
data['Completion'] = data.apply(lambda row: f"{process_list_data(row['fortune_telling'])} Keywords: {process_list_data(row['keywords'])}. Light meanings: {process_list_data(row['meanings.light'])}. Shadow meanings: {process_list_data(row['meanings.shadow'])}. Questions to consider: {process_list_data(row['Questions to Ask'])}", axis=1)

# Example of how data is transformed
data[['Prompt', 'Completion']].head()


,Prompt,Completion
0,Tell me about the tarot card The Fool.,Watch for new projects and new beginnings Prep...
1,Tell me about the tarot card The Magician.,A powerful man may play a role in your day You...
2,Tell me about the tarot card The High Priestess.,A mysterious woman arrives A sexual secret may...
3,Tell me about the tarot card The Empress.,Pregnancy is in the cards An opportunity to be...
4,Tell me about the tarot card The Emperor.,A father figure arrives A new employer or auth...


In [ ]:
import json

# Convert the DataFrame to JSON Lines format
formatted_data = data[['Prompt', 'Completion']].to_dict(orient='records')
jsonl_content = [json.dumps(record) for record in formatted_data]

# Save to a new JSONL file
jsonl_file_path = '/content/drive/MyDrive/2656 ML Project/archive/tarot_data_for_finetuning.jsonl'
with open(jsonl_file_path, 'w') as outfile:
    for entry in jsonl_content:
        outfile.write(entry + '\n')


In [ ]:
import json

# Path to your JSONL file
jsonl_file_path = '/content/drive/MyDrive/2656 ML Project/archive/tarot_data_for_finetuning.jsonl'

# Open the file and read it line by line
with open(jsonl_file_path, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        print(json_data)
        # You can break after the first few lines if you just want a preview
        # brea

{'Prompt': 'Tell me about the tarot card The Fool.', 'Completion': 'Watch for new projects and new beginnings Prepare to take something on faith Something new comes your way; go for it Keywords: freedom faith inexperience innocence. Light meanings: Freeing yourself from limitation Expressing joy and youthful vigor Being open-minded Taking a leap of faith Attuning yourself to your instincts Being eager or curious Exploring your potential Embracing innovation and change. Shadow meanings: Being gullible and naive Taking unnecessary risks Failing to be serious when required Being silly or distracted Lacking experience Failing to honor well-established traditions and limits Behaving inappropriately. Questions to consider: What would I do if I felt free to take a leap? How willing am I to be vulnerable and open? How might past experiences help in this new situation?'}
{'Prompt': 'Tell me about the tarot card The Magician.', 'Completion': 'A powerful man may play a role in your day Your curre

In [ ]:
import json

def transform_data(input_path, output_path):
    with open(input_path, 'r') as file, open(output_path, 'w') as output_file:
        for line in file:
            data = json.loads(line)
            prompt = data['Prompt']
            completion = data['Completion']

            # Creating a new structured format for the conversation
            transformed_entry = {
                "messages": [
                    {"role": "system", "content": "You are a tarot reader. Please interpret the card mentioned by the user."},
                    {"role": "user", "content": prompt},
                    {"role": "assistant", "content": completion}
                ]
            }

            # Write the transformed entry to the new JSONL file
            output_file.write(json.dumps(transformed_entry) + '\n')

# Specify the path to your existing file and the new file to create
input_file_path = "/content/drive/MyDrive/2656 ML Project/archive/tarot_data_for_finetuning.jsonl"
output_file_path = '/content/drive/MyDrive/2656 ML Project/archive/transformed_tarot_data_finetune.jsonl'

# Call the function to transform the data
transform_data(input_file_path, output_file_path)


In [ ]:
import json

# Path to your JSONL file
jsonl_file_path = '/content/drive/MyDrive/2656 ML Project/archive/transformed_tarot_data_finetune.jsonl'

# Open the file and read it line by line
with open(jsonl_file_path, 'r') as file:
    for line in file:
        json_data = json.loads(line)
        print(json_data)
        # You can break after the first few lines if you just want a preview
        # brea

{'messages': [{'role': 'system', 'content': 'You are a tarot reader. Please interpret the card mentioned by the user.'}, {'role': 'user', 'content': 'Tell me about the tarot card The Fool.'}, {'role': 'assistant', 'content': 'Watch for new projects and new beginnings Prepare to take something on faith Something new comes your way; go for it Keywords: freedom faith inexperience innocence. Light meanings: Freeing yourself from limitation Expressing joy and youthful vigor Being open-minded Taking a leap of faith Attuning yourself to your instincts Being eager or curious Exploring your potential Embracing innovation and change. Shadow meanings: Being gullible and naive Taking unnecessary risks Failing to be serious when required Being silly or distracted Lacking experience Failing to honor well-established traditions and limits Behaving inappropriately. Questions to consider: What would I do if I felt free to take a leap? How willing am I to be vulnerable and open? How might past experienc

In [ ]:
response = client.files.create(
  file=open("/content/drive/MyDrive/2656 ML Project/archive/transformed_tarot_data_finetune.jsonl", "rb"),  # Ensure the file path matches where your JSONL file is stored
  purpose="fine-tune"
)

In [ ]:
file_id = response.id  # Store the file ID for use in the fine-tuning request
print("Uploaded file ID:", file_id)

Uploaded file ID: file-DgkXaiME44TOEKzUG4zJm7lX


In [ ]:
fine_tune_response = client.fine_tuning.jobs.create(
    training_file=file_id,
    model="gpt-3.5-turbo"
    )

In [ ]:
print("Fine-tune job created with ID:", fine_tune_response.id)


Fine-tune job created with ID: ftjob-Q1lFbr5szdWsp0dqZsIyMJNs


In [ ]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Q1lFbr5szdWsp0dqZsIyMJNs', created_at=1714187154, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-uJoG9flkzxDwirFjoYGxsfu2', result_files=[], seed=1854446415, status='running', trained_tokens=None, training_file='file-DgkXaiME44TOEKzUG4zJm7lX', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None), FineTuningJob(id='ftjob-qANlm1igJWYOiOFjphZWj2P7', created_at=1714176286, error=Error(code='invalid_training_file', message='The job failed due to an invalid training file. Unexpected file format, expected either prompt/completion pairs or chat messages.', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_r

In [ ]:
# Retrieve the state of a fine-tune
status_response = client.fine_tuning.jobs.retrieve(fine_tune_response.id)
print("Fine-tune job status:", status_response.status)


Fine-tune job status: succeeded


In [ ]:
status_response


FineTuningJob(id='ftjob-Q1lFbr5szdWsp0dqZsIyMJNs', created_at=1714187154, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9ISnjarY', finished_at=1714187642, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-uJoG9flkzxDwirFjoYGxsfu2', result_files=['file-l8JINdkmA34mBWgtilOgi3hS'], seed=1854446415, status='succeeded', trained_tokens=47241, training_file='file-DgkXaiME44TOEKzUG4zJm7lX', validation_file=None, integrations=[], user_provided_suffix=None, estimated_finish=None)

In [ ]:
status_response.fine_tuned_model

'ft:gpt-3.5-turbo-0125:personal::9ISnjarY'

In [ ]:
result = client.chat.completions.create(
    model='ft:gpt-3.5-turbo-0125:personal::9ISnjarY',
    messages=[
        {"role": "system", "content": "You are a tarot reader. Please interpret the cards drawn for the user."},
        {"role": "user", "content": "Tell me about Hermit"}
    ]
)

print(result.choices[0].message.content)


All your questions can be answered within, if you look "up there" Getting to the light at the end of the tunnel is simply a matter of time Keywords: insight tranquility wisdom solitude awareness clarity Contentment realism. Light meanings: Taking "alone time" for yourself Meditating or engaging in some other form of solitary contemplation Allowing events to unfold in their own time Gaining insight. Shadow meanings: Hiding from the truth Refusing to be honest with yourself or others Blaming your problems on others or the world Concealing something from prying eyes. Questions to consider: What can you learn by standing apart from others? How difficult is it for you to remain patient? To offer extended or unconditional forgiveness? How can you achieve a real breakthrough?
